In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.stats as st
import pickle
import utils
import moran_auto
import gmm
import kmeans_utils
import figure_functions
from sklearn.preprocessing import MinMaxScaler
from sklearn import mixture
from sklearn.cluster import KMeans
import csv
from scipy.misc import imread
from collections import defaultdict
from matplotlib.patches import Ellipse
from matplotlib.patches import Polygon
import glob
from pandas.tseries.holiday import USFederalHolidayCalendar
import datetime
import gmplot
import matplotlib.cm as cm
import run_all

In [4]:
curr_dir = os.getcwd()
data_path = curr_dir + '/../data'
fig_path = curr_dir + '/../figs'
results_path = curr_dir + '/../results'
animation_path = curr_dir + '/../animation'
belltown_path = data_path + '/Belltown_Hour'
commcore_path = data_path + '/CommercialCore_Hour'
pikepine_path = data_path + '/PikePine_Hour'
dennytriangle_path = data_path + '/DennyTriangle_Hour'
firsthill_path = data_path + '/FirstHill_Hour'
raw_transaction_path = data_path + '/RawTransactionData'

In [5]:
pd.set_option('display.max_columns', None)  

In [6]:
block_info = pd.read_csv(os.path.join(data_path, 'block_info.csv'))
block_info = block_info.loc[block_info['PaidParkingArea'] == 'Belltown']
block_keys = block_info['ElementKey'].unique().tolist()

paystation_info = pd.read_csv(os.path.join(data_path, 'paystation_info.csv'))
paystation_info = paystation_info.loc[paystation_info['PAIDAREA'] == 'Belltown']
pay_keys = paystation_info['ELMNTKEY'].unique().tolist()

In [7]:
element_keys, loads, gps_loc, park_data, N, P, idx_to_day_hour, day_hour_to_idx = utils.load_data_new(data_path, [belltown_path])

Skipping block 13294 because the supply is always 0.
Skipping block 13517 because the supply is always 0.
Skipping block 27278 because the supply is always 0.
Skipping block 31490 because the occupancy is always 0.
Skipping block 32021 because the supply is always 0.
Skipping block 32025 because the supply is always 0.
Skipping block 32029 because the supply is always 0.
Skipping block 49658 because the supply is always 0.
Skipping block 54674 because the supply is always 0.
Skipping block 58805 because the supply is always 0.
Skipping block 68918 because the supply is always 0.
Skipping block 70745 because the supply is always 0.
Skipping block 70749 because the occupancy is always 0.
Skipping block 72325 because the supply is always 0.
Skipping block 72326 because the supply is always 0.
Skipping block 77117 because the supply is always 0.
Skipping block 77121 because the supply is always 0.
Skipping block 77350 because the supply is always 0.
Skipping block 78114 because the supply 

In [8]:
results = gmm.locational_demand_analysis(park_data, gps_loc, N)

In [9]:
results

[25.44,
 17.81,
 11.88,
 11.65,
 11.13,
 12.55,
 14.87,
 12.92,
 11.58,
 12.88,
 12.97,
 12.95,
 24.94,
 18.22,
 15.01,
 11.78,
 8.99,
 9.83,
 11.99,
 12.44,
 12.84,
 13.3,
 12.13,
 12.88,
 25.38,
 18.09,
 10.89,
 10.87,
 10.7,
 10.25,
 13.3,
 9.67,
 10.49,
 11.44,
 12.25,
 12.59,
 24.43,
 16.62,
 11.4,
 11.01,
 10.56,
 10.16,
 12.95,
 12.1,
 13.98,
 13.51,
 13.46,
 13.09,
 22.73,
 15.31,
 9.73,
 9.77,
 10.07,
 9.23,
 13.35,
 12.11,
 12.18,
 11.5,
 11.75,
 11.13,
 24.62,
 25.3,
 17.21,
 14.86,
 15.27,
 11.94,
 15.54,
 14.18,
 14.8,
 9.55,
 10.66,
 11.03]

In [ ]:
for row in np.array(results).reshape((6, 12)):
    print row

In [ ]:
d = np.array([[ 23.48,  19.13,  14.15,   9.08,  10.93,  12.27,  13.94,  11.72,  11.59,  14.59,  13.79,  12.92],
    [ 25.73,  18.6 ,  14.21,  12.26,   9.41,  11.05,  12.68,  13.19,  14.17,  13.33,  12.04,  12.04],
    [ 25.15,  17.62,  11.84,   9.92,  10.56,  11.45,  13.4 ,  11.35,   9.57,  11.  ,  11.64,  14.97],
    [ 24.67,  16.9 ,  11.27,   9.47,   9.43,  10.64,  13.16,  12.24,  14.35,  13.9 ,  13.25,  13.59],
    [ 22.96,  13.67,   8.91,   9.49,   8.3 ,   9.73,  12.87,  12.78,  12.17,  11.61,  11.49,  11.13],
    [ 24.96,  23.9 ,  19.16,  14.44,  13.53,  11.16,  15.79,  13.53,  13.71,  9.7  ,  9.92 ,  11.11]])

In [ ]:
c = np.array([[ 28.43,  17.98,   9.57,   9.55,   9.33,   8.62,  10.57,  12.54,  11.82,  12.24,  13.69,  12.48],
[ 30.65,  18.94,  12.52,  10.92,   9.47,   8.21,  11.03,  12.81,  11.41,  11.23,  12.41,  11.34],
[ 33.74,  18.82,   9.04,   7.7 ,   7.86,  10.35,  11.67,  11.23,  11.96,  10.71,  12.18,  13.79],
[ 29.88,  14.44,   7.58,   8.32,   9.56,  10.  ,   9.96,  11.47,  11.54,  11.67,  12.63,  11.02],
[ 26.84,  14.94,  10.8 ,   7.75,   9.76,   8.79,  10.77,   9.86,  11.99,  12.06,  12.89,  10.73],
[ 35.81,  25.91,  21.16,  16.46,  14.8 ,  13.38,  14.  ,  13.74,  13.24,  11.97,  10.27,  10.3 ]])

In [ ]:
a = np.array([[30.51,  18.88,   9.71,   8.59,   9.09,   8.29,  10.63,  14.07,  12.16,  12.47,  12.66,  11.66],
[ 35.27,  18.96,  12.27,  12.25,   8.64,   8.53,  10.67,  12.07,  11.82,  12.54,  11.61,  11.22],
[ 31.88,  18.46,  10.11,   7.88,   8.38,  10.43,  11.91,  11.12,  10.49,  11.2 ,  12.64,  13.98],
[ 30.77,  15.34,   8.62,   7.95,   9.54,   9.9 ,  10.25,  11.5 ,  11.65,  12.31,  11.82,  11.23],
[ 29.31,  15.06,   9.54,   8.05,  10.29,   9.33,  11.36,   9.88,  11.22,  12.1 ,  13.41,  10.45],
[ 36.23,  28.43,  20.2 ,  16.99,  12.94,  12.55,  11.2 ,  11.16,  12.23,  12.72,  10.1 ,  10.25]])

In [ ]:
b = np.array([[ 27.49,  16.57,  11.64,  10.52,   9.82,  10.18,  11.32,  10.43,  12.4 ,  11.52,  14.78,  13.88],
[ 29.72,  17.32,  13.34,  11.12,   9.48,   8.43,  11.39,  11.65,  12.77,  11.8 ,  10.84,  12.6 ],
[ 28.4 ,  16.88,  10.33,   9.05,   9.92,  10.23,  11.24,  10.17,  10.75,  10.07,  12.32,  13.53],
[ 29.98,  13.36,   8.28,   8.93,   8.97,   8.69,  10.05,  11.47,  11.7 ,  12.23,  12.6 ,  13.01],
[ 25.9 ,  14.95,  10.92,   9.63,   8.41,   9.24,   9.8 ,   9.16,  10.96,  11.83,  12.48,  11.58],
[ 34.22,  26.09,  22.49,  17.16,  16.37,  16.05,  18.08,  16.41,  14.73,  12.48,  10.87,  11.98]])

In [ ]:
(a - d).sum()

In [ ]:
gps_loc_old, loads_old, park_data_old, N_old, P_old, idx_to_day_hour_old, day_hour_to_idx_old = utils.load_data(data_path)

In [ ]:
time = 58

In [ ]:
run_all.run_figures(element_keys, loads, gps_loc, N, P, data_path, fig_path, animation_path, time)

In [ ]:
mask_small = True

time = idx_to_day_hour[0]
day = time[0]
hour = time[1]

data_df = park_data.loc[(park_data['Day'] == day) & (park_data['Hour'] == hour)]
block_keys = sorted(data_df.index.get_level_values(1).unique().tolist())

average_accuracies = []
centers = [] 

for train_time in data_df.index.get_level_values(0).unique().tolist():
    train_data = data_df.xs(train_time, level=0)

    if mask_small:
        train_data.loc[(train_data['Load'] <= .05), 'Load'] = np.nan

    train_data = train_data.dropna()

    train_active_index = train_data.index.tolist()
    train_mask = [element_keys.index(train_active_index[i]) for i in xrange(len(train_active_index))]  
    train_to_label_map = {train_active_index[i]: i for i in xrange(len(train_active_index))}

    train_loads = train_data['Load'].values.reshape((-1, 1))

    train = np.hstack((train_loads, gps_loc[train_mask]))

    # Saving the scaling so it can be applied to the test set as well.
    unscaled_loads = train[:, 0]    
    scaler = MinMaxScaler().fit(train)
    train = scaler.transform(train)

    gmm = mixture.GaussianMixture(n_init=10, n_components=4, 
                                  covariance_type='diag').fit(train)

    # Scaling the mean and covariances back to GPS coordinates.
    means = np.vstack(([(mean[1:] - scaler.min_[1:])/(scaler.scale_[1:]) for mean in gmm.means_]))
    covs = np.dstack(([np.diag((cov[1:])/(scaler.scale_[1:]**2)) for cov in gmm.covariances_])).T

    centers.append(means)

    train_labels = gmm.predict(train)

    accuracies = []

    # For each other day of data, predict using model that was fit.
    for test_time in data_df.index.get_level_values(0).unique().tolist():

        # Skipping predicting on the time that was trained on.
        if test_time == train_time:
            continue

        test_data = data_df.xs(test_time, level=0)

        if mask_small:
            test_data.loc[(test_data['Load'] <= .05), 'Load'] = np.nan

        test_data = test_data.dropna()

        test_data = test_data.loc[test_data.index.isin(train_active_index)]

        test_active_index = test_data.index.tolist()
        test_mask = [element_keys.index(test_active_index[i]) for i in xrange(len(test_active_index))]  

        test_loads = test_data['Load'].values.reshape((-1, 1))

        test = np.hstack((test_loads, gps_loc[test_mask]))

        test = scaler.transform(test)

        # Assigning labels for the test points using the model that was trained. 
        test_labels = gmm.predict(test)

        correct = 0
        incorrect = 0

        for i in xrange(len(test_labels)):
            key = test_active_index[i]
            idx = train_to_label_map[key]

            if test_labels[i] == train_labels[idx]:
                correct += 1
            else:
                incorrect += 1

        # A prediction is deemed correct if a block keeps the same label.
        accuracy = correct/float(correct + incorrect)
        accuracies.append(accuracy)

        # Getting average prediction accuracy over all test sets.
    average_accuracies.append(np.array(accuracies).mean())

# Average error for the particular day and hour combination.
#time_avg_error = round(100.0 - np.array(average_accuracies).mean() * 100, 2)

In [ ]:
(train_data['Load'] <= .05).values

In [ ]:
train_data.loc[(train_data['Load'].values <= .05), 'Load'] = np.nan

In [ ]:
train_data

In [ ]:
train_data.loc[(train_data['Load'] <= .05), 'Load'] = np.nan

In [ ]:
average_accuracies

In [ ]:
block_info.loc[block_info['ElementKey'] == 76990]

In [ ]:
train_active_index

In [ ]:
test_active_index

In [ ]:
check.index.isin(train_active_index)